In [ ]:
!pip3 install flotorch-eval crewai plotly opentelemetry-api opentelemetry-sdk pandas ragas duckduckgo-search openlit langchain_aws


In [ ]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../..")))

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.sdk.trace.export.in_memory_span_exporter import InMemorySpanExporter


# Create an in-memory span exporter
memory_exporter = InMemorySpanExporter()
span_processor = SimpleSpanProcessor(memory_exporter)

# Set up the tracer provider and add the span processor
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(span_processor)
trace.set_tracer_provider(tracer_provider)

In [ ]:
import openlit

# Initialize OpenLit - this will automatically instrument CrewAI when it's imported
openlit.init()

In [ ]:
from crewai.tools import tool
from duckduckgo_search import DDGS

@tool('DuckDuckGoSearch')
def search_tool(search_query: str):
    """Search the web for information on a given topic"""
    return DDGS().text(search_query, max_results=5)

@tool('SalesforceIntegration')
def salesforce_tool(soql_query: str):
    """Call Salesforce API to get data"""
    return "Salesforce Integration"

from crewai import LLM

model = LLM(
    # model="sagemaker/INSERT ENDPOINT NAME",
    model="bedrock/us.amazon.nova-pro-v1:0",
    temperature=0.7, max_tokens=4*1024,
)

from crewai import Agent, Task, Crew


writer = Agent(
        role="Writer",
        goal="You make GenAI concepts understandable for newbies exploring GenAI on AWS",
        backstory="You're an expert in writing crisp summaries about GenAI on AWS.",
        tools=[search_tool],
        llm=model
    )

task = Task(description=("What is {topic}?"),
            expected_output=("Compose a short summary that includes the answer."),
            agent=writer)

crew = Crew(
  agents=[writer],
  tasks=[task],
  share_crew=False
)

result = crew.kickoff({"topic": "AWS Bedrock"})
print(result)



In [ ]:
spans = memory_exporter.get_finished_spans()

print("Number of spans:", len(spans))

In [ ]:
from langchain_aws import ChatBedrockConverse
from ragas.llms import LangchainLLMWrapper

region = "us-east-1"

bedrock_model = ChatBedrockConverse(
    region_name=region,
    endpoint_url=f"https://bedrock-runtime.us-east-1.amazonaws.com",
    model_id="us.amazon.nova-micro-v1:0"
)

llm = LangchainLLMWrapper(bedrock_model)



In [ ]:
from flotorch_eval.agent_eval.core.converter import TraceConverter

def create_trajectory(spans):
    converter = TraceConverter()
    trajectory = converter.from_spans(spans)
    return trajectory


In [ ]:
trajectory = create_trajectory(spans)
trajectory

In [ ]:
from flotorch_eval.agent_eval.core.evaluator import Evaluator
from flotorch_eval.agent_eval.metrics.langchain_metrics import (
    TrajectoryEvalWithLLMMetric,
    TrajectoryEvalWithoutLLMMetric,
)
from flotorch_eval.agent_eval.metrics.latency_metrics import LatencyMetric
# from flotorch_eval.agent_eval.metrics.ragas_metrics import (
#     AgentGoalAccuracyMetric,
#     ToolCallAccuracyMetric,
# )
from flotorch_eval.agent_eval.metrics.base import MetricConfig
from flotorch_eval.agent_eval.metrics.usage_metrics import UsageMetric


from flotorch_eval.agent_eval.metrics_new.agent_goal_accuracy import AgentGoalAccuracyMetric
from flotorch_eval.agent_eval.metrics_new.tool_call_accuracy import ToolCallAccuracyMetric

evaluator = Evaluator([
    ToolCallAccuracyMetric(),
    AgentGoalAccuracyMetric(llm=llm,config=MetricConfig(
        metric_params={
            "reference_answer": "Amazon Bedrock is a fully managed service that makes it easy to use foundation models from third-party providers and Amazon. It allows users to build generative AI applications with a choice of foundation models from different AI companies, using a single API. Users can customize these models with their data, orchestrate multistep tasks, trace reasoning, and apply guardrails for responsible AI. Additionally, Amazon Bedrock enables the creation of generative AI workflows by connecting its features with other AWS services."  # optional
        }
    )),
    TrajectoryEvalWithLLMMetric(llm = bedrock_model,config=MetricConfig(
        metric_params={
            "reference_outputs": [
                {"role": "user", "content": "What is AWS Bedrock?"},
                {
                    "role": "assistant",
                    "content": "To compose a poem about Amazon Bedrock, I first need to gather information about what Amazon Bedrock is. I will use the available tool to search for this information.",
                    "tool_calls": [
                        {
                            "function": {
                                "name": "Search the web for information on a given topic",
                                "arguments": "{\"search_query\": \"Amazon Bedrock\"}"
                            }
                        }
                    ]
                },
                {"role": "tool", "content": "{\"searchParameters\": {\"q\": \"Amazon Bedrock\", \"type\": \"search\", \"num\": 5, \"engine\": \"google\"}, \"organic\": [{\"title\": \"Amazon Bedrock - Generative AI - AWS\", \"link\": \"https://aws.amazon.com/bedrock/\", \"snippet\": \"Amazon Bedrock Data Automation streamlines the generation of valuable insights from unstructured multimodal content such as documents, images, audio, and videos ...\", \"position\": 1, \"sitelinks\": [{\"title\": \"Amazon Bedrock\", \"link\": \"https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html\"}, {\"title\": \"Amazon Bedrock Pricing\", \"link\": \"https://aws.amazon.com/bedrock/pricing/\"}, {\"title\": \"Amazon Bedrock Documentation\", \"link\": \"https://docs.aws.amazon.com/bedrock/\"}, {\"title\": \"Amazon Bedrock FAQs\", \"link\": \"https://aws.amazon.com/bedrock/faqs/\"}, {\"title\": \"Amazon Bedrock Agents\", \"link\": \"https://aws.amazon.com/bedrock/agents/\"}]}, {\"title\": \"Getting Started with Amazon Bedrock - AWS\", \"link\": \"https://aws.amazon.com/awstv/watch/6ff4cd6fa97/\", \"snippet\": \"So check the region that you're currently in, make sure it's a region that's supported by Bedrock. Then I'm gonna scroll to the bottom of this ...\", \"position\": 2}], \"relatedSearches\": [{\"query\": \"Amazon Bedrock pricing\"}, {\"query\": \"Amazon Bedrock documentation\"}, {\"query\": \"Amazon Bedrock Claude\"}, {\"query\": \"Amazon Bedrock logo\"}, {\"query\": \"Amazon Bedrock DeepSeek\"}], \"credits\": 1}"},
                {"role": "assistant", "content": "Based on the observation, I have learned that mazon Bedrock is a fully managed service that makes it easy to use foundation models from third-party providers and Amazon. It allows users to build generative AI applications with a choice of foundation models from different AI companies, using a single API. Users can customize these models with their data, orchestrate multistep tasks, trace reasoning, and apply guardrails for responsible AI. Additionally, Amazon Bedrock enables the creation of generative AI workflows by connecting its features with other AWS services."}
            ]
        }
    )),
    UsageMetric(config=MetricConfig(
        metric_params={"aws_region": "us-east-1"}
    )),
    LatencyMetric()
])

# Evaluate trajectory
results = await evaluator.evaluate(trajectory)



In [ ]:
results.scores